In [ ]:
       if len(code) <= 5:
            response = None
            error = "length of source code is too short to generate a response"
            error_type = "not_enough_tokens"

        elif len(code) > 5:

            #code = self.code_clean(code)
            prompt_template = PromptTemplate.from_template(
            '''
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            "{code}"

            ####

            Expected Output Format:
            // Test case: Testing....
            <code>
            .
            .

            ''')
            
            system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Javascript source code as input and you need to generate unit test cases in Jasmine framework as output "
           
            prompt = prompt_template.format(code = code)

            input_prompt_size = self.num_tokens_from_string(
                prompt, "cl100k_base")
            print("input prompt size: " + str(input_prompt_size))
            azure_start_time = datetime.now()
            if input_prompt_size < 1000:
                model_used = 'gpt-35-turbo'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)
                if len(matching_docs) == 0:
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "user", "content": quote(prompt)}] 
                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.client.chat.completions.create(
                        model=model_used,
                        messages=value
                        )
                else:
                    matching_docs = self.restructure_document_object(matching_docs)
                    examples_retreived = self.few_shot_prompting_template(matching_docs)
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "system", "content": quote(examples_retreived)},
                             {"role": "user", "content": quote(prompt)}]

                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.client.chat.completions.create(
                        model=model_used,
                        messages=value
                        )


            elif input_prompt_size >= 1000 and input_prompt_size < 4000:
                model_used = 'gpt-35-turbo-16k'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)
                
                if len(matching_docs) == 0:
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "user", "content": quote(prompt)}] 
                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.client.chat.completions.create(
                        model=model_used,
                        messages=value
                        )
                else:
                    matching_docs = self.restructure_document_object(matching_docs)
                    examples_retreived = self.few_shot_prompting_template(matching_docs)
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "system", "content": quote(examples_retreived)},
                             {"role": "user", "content": quote(prompt)}]

                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.client.chat.completions.create(
                        model=model_used,
                        messages=value
                        )
                    

In [ ]:

python
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=value
)